In [ ]:
from selenium import webdriver
import re
import time
# from datetime import datetime
import os
from tqdm import tqdm
# import pycurl
import codecs
import base64
import pandas as pd

colName = ['operationStatus', 'registrationCode', 'name', 'type', 'administrator', 'certTime', 'registrationBereau', 'representative', 'date', 'funding', 'scope', 'headquater']

dataCollection = {}

keyword = '肿瘤'
for i in colName:
    dataCollection[i] = []
    
for file in os.listdir(f'/Users/alan/Desktop/keywords/keyword={keyword}'):
    if 'html' in file:
        f = codecs.open(f'/Users/alan/Desktop/keywords/keyword={keyword}/{file}', 'r')
        test = f.read()
        result = re.findall('<span data-v-6e51d4b3="">统一社会信用代码:\n                        <span data-v-6e51d4b3="">\n                    <span>.*</span>', test)
        urlList = []
        for i in range(0, len(result)):
            result[i] = '{"id":"' + result[i][-25:-7] + '"}'
            base64Encoded = base64.b64encode(bytes(result[i], 'utf-8'), altchars=None).decode("utf-8")
            urlList.append(f'https://xxgs.chinanpo.mca.gov.cn/gsxt/newDetails?b={base64Encoded}')

        for i in tqdm(range(0, len(urlList))):
            driverPath = '/Users/alan/Downloads/chromedriver'
            driver = webdriver.Chrome(driverPath) 
            driver.get(urlList[i])
            time.sleep(1)
            response = driver.page_source

            try:
                operationStatus = re.findall(r'[\u4e00-\u9fff]+', re.findall('class="ant-tag ant-tag-has-color" style="background-color:.*</div></div><div data-v-283f9344="" class="tag_center">', response)[0])
            except:
                operationStatus = None
            try:
                name = re.findall('<span data-v-d14c252c="" class="data_break_line">\n.*\n', re.findall('社会组织名称\n.*社会组织类型\n', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[1].strip()
            except:
                name = None
            try:
                type = re.findall('<span data-v-d14c252c="" class="data_break_line">\n.*\n', re.findall('社会组织类型\n.*业务主管单位\n ', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[1].strip()
            except:
                type = None
            try:
                administrator = re.findall('<span data-v-d14c252c="" class="data_break_line">\n.*\n', re.findall('业务主管单位\n.*证书有效期\n', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[1].strip()
            except:
                administrator = None
            try:
                certTime = re.findall('<span data-v-d14c252c="" class="data_break_line">\n.*\n', re.findall('证书有效期\n.*登记管理机关\n', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[1].strip()
            except:
                certTime = None
            try:
                registrationBereau = re.findall('<span data-v-d14c252c="" class="data_break_line">\n.*\n', re.findall('登记管理机关\n.*法定代表人\n', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[1].strip()
            except:
                registrationBereau = None
            try:
                representative = re.findall('<span data-v-d14c252c="" class="data_break_line">\n.*\n', re.findall('法定代表人\n.*成立登记日期\n', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[1].strip()
            except:
                representative = None
            try:
                date = re.findall('<span data-v-d14c252c="" class="data_break_line">\n.*\n', re.findall('成立登记日期\n.*注册资金', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[1].strip()
            except:
                date = None
            try:
                funding = re.findall(r'[\d]+', re.findall('<span data-v-d14c252c="" class="data_break_line">\n.*\n', re.findall('注册资金.*业务范围\n', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[1].strip())[0] + \
                            re.findall(r'[\u4e00-\u9fff]+', re.findall('<span data-v-d14c252c="" class="data_break_line">\n.*\n', re.findall('注册资金.*业务范围\n', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[1].strip())[0]
            except:
                funding = None
            try:
                scope = re.findall(r'[\u4e00-\u9fff]+', re.findall('span data-v-d14c252c="" class="text long_text2"><span data-v-d14c252c="" class="aaae0122Txt">\n.*\n', re.findall('业务范围\n.*住所\n', response, re.DOTALL)[0], re.DOTALL)[0])[:-1]
            except:
                scope = None
            try:
                headquarter = re.findall('span data-v-d14c252c="" class="text long_text2"><span data-v-d14c252c="" class="aaae0122Txt">\n.*\n', re.findall('住所\n.*点击或者下拉加载更多', response, re.DOTALL)[0], re.DOTALL)[0].split('\n')[-2].strip()
            except:
                headquarter = None
            this_collection = {
                                'operationStatus': operationStatus,
                                'registrationCode': result[i][7:-2],
                                'name': name,
                                'type': type,
                                'administrator': administrator,
                                'certTime': certTime,
                                'registrationBereau': registrationBereau,
                                'representative': representative,
                                'date': date,
                                'funding': funding,
                                'scope': scope,
                                'headquater': headquarter
                            }
            for i in colName:
                dataCollection[i].append(this_collection[i])

            driver.close()

df = pd.DataFrame(data = dataCollection)
df.to_csv(f'/Users/alan/Desktop/data_{keyword}.csv', index = None, encoding = 'utf-8-sig')

In [ ]:
import pandas as pd
import requests
from requests.exceptions import ConnectionError, Timeout, RequestException
from bs4 import BeautifulSoup
import warnings
from urllib3.exceptions import InsecureRequestWarning
import time
import random

# Suppress InsecureRequestWarnings
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# Load the CSV file
file_path = 'data_source_1011_161_entries.csv'
data = pd.read_csv(file_path)

# Extract non-empty URLs from the 'Source URL' column
urls = data['Source URL'].dropna().unique()

# Define a list of user-agents to simulate different browsers
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1'
]

# Define headers for requests
headers = {'User-Agent': random.choice(user_agents)}

# Create a session to reuse connections
session = requests.Session()

# Function to scrape a web page and extract information
def scrape_url(url, retries=3, delay=5, timeout=10):
    attempt = 0
    while attempt < retries:
        try:
            response = session.get(url, headers=headers, timeout=timeout, verify=False)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'lxml')

            # Extract title
            title = soup.title.string if soup.title else "No title"

            # Extract headings
            headings = [h.get_text(strip=True) for h in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]

            # Extract paragraphs
            paragraphs = [p.get_text(strip=True) for p in soup.find_all('p')]

            # Extract links
            links = [a['href'] for a in soup.find_all('a', href=True)]

            # Extract meta description
            meta_description = soup.find('meta', attrs={'name': 'description'})
            description = meta_description['content'] if meta_description else "No meta description"

            # Extract images
            images = [img['src'] for img in soup.find_all('img', src=True)]

            return {
                "Title": title,
                "Headings": headings,
                "Paragraphs": paragraphs,
                "Links": links,
                "Description": description,
                "Images": images
            }

        except (ConnectionError, Timeout, RequestException) as e:
            attempt += 1
            if attempt < retries:
                print(f"Retrying {url}, attempt {attempt}/{retries} after {delay} seconds.")
                time.sleep(delay)
            else:
                return f"Failed to retrieve {url}: {e}"
        except Exception as e:
            return f"Parsing failed for {url}: {e}"

# Scrape each URL and store the results
scraped_data = []
for url in urls:
    info = scrape_url(url)
    scraped_data.append((url, info))

# Convert scraped data to a DataFrame for display
scraped_df = pd.DataFrame(scraped_data, columns=['URL', 'Website Information'])

# Display the scraped data
print(scraped_df)
